In [1]:
#pip install PyPDF2 pdfplumber pandas openpyxl

In [2]:
#pip install pdfplumber pytesseract pillow pdf2image

In [3]:
#/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"


In [4]:
import pdfplumber
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pytesseract
from pdf2image import convert_from_path
from urllib.parse import urlparse
from PIL import Image

In [5]:
#wasnt able to get ecobank ('Ecobank_URL' : 'https://www.ecobank.com/group/investor-relations/key-figures/performance')
banks_urls = {'CBG_URL' :'https://cbg.com.gh/annual-report.html',
'GT_URL': 'https://www.gtbghana.com/about/financial-information',
'FBN_URL' :'https://www.fbnbankghana.com/our-financials/',
'First_National_URL' : 'https://www.firstnationalbank.com.gh/news-insight/financial-reports.html',
'Cal_URL' : 'https://www.annualreportsghana.com/gse/calbank/',
'SG_URL' : 'https://societegenerale.com.gh/en/your-bank/investor-relations/annual-reports/',
'Stanbic_URL' :'https://www.stanbicbank.com.gh/gh/personal/about-us/financials',
'Zenith_URL' : 'https://www.zenithbank.com.gh/about-us/financial-report/#annual-report',
'Fidelity_URL' : 'https://www.fidelitybank.com.gh/downloadables/financial-reports',
'Ecobank_URL': 'https://www.annualreportsghana.com/gse/ecobank-ghana/'}

In [6]:
# Dictionary to store the responses
bank_html_content = {}

for bank, url in banks_urls.items():
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text,'html.parser')
            # Store the content in the dictionary
            links = soup.find_all('a')
            hrefs = [link.get('href') for link in links if link.get('href')]
            bank_html_content[bank] = hrefs
        else:
            print(f"Failed to fetch data for {bank}. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Error fetching data for {bank}: {e}")

# Access content for a specific bank
#print(bank_html_content.get("Consolidated Bank Ghana", "No data available"))

In [7]:
#didnt get stanbic
bank_html_content

{'CBG_URL': ['personal.html',
  'business.html',
  'services.html',
  'digital.html',
  'https://selfserviceportal.apps.cbg.com.gh',
  'cbg-atms---branches.html',
  'atms-location.html',
  'https://app.purechat.com/w/cbglivechat',
  'index.html',
  'index.html',
  'about-cbg.html',
  'about-cbg.html',
  'board-of-directors.html',
  'executive-management.html',
  'personal.html',
  'business.html',
  'digital.html',
  'services.html',
  'https://cbg.com.gh/news/',
  'sustainability-commitment-statement.html',
  'contact-us.html',
  'assets/cbg-tariff-guide-update-july-2024-website-.pdf',
  'https://cbg.com.gh/rates.html',
  'https://ibank.cbg.com.gh/EPS_INT_CORP/',
  'https://ibank.cbg.com.gh/',
  'assets/2024-q3-unaudited-financial-statements.pdf',
  'assets/2024-q3-unaudited-financial-statements.pdf',
  'assets/2024-q3-unaudited-financial-statements.pdf',
  'assets/2024-q3-unaudited-financial-statements.pdf',
  'assets/2024-q2-unaudited-financial-statements.pdf',
  'assets/2024-q2-una

In [8]:
financial_links = {}

for bank, href_link in bank_html_content.items():
    # Filter links that contain 'Financial' or 'financial'
    filtered_links = [link for link in href_link if 'financial' in link.lower()]
    
    # Add to the new dictionary if there are any filtered links
    if filtered_links:
        financial_links[bank] = filtered_links

In [9]:
financial_links

{'CBG_URL': ['assets/2024-q3-unaudited-financial-statements.pdf',
  'assets/2024-q3-unaudited-financial-statements.pdf',
  'assets/2024-q3-unaudited-financial-statements.pdf',
  'assets/2024-q3-unaudited-financial-statements.pdf',
  'assets/2024-q2-unaudited-financial-statements.pdf',
  'assets/2024-q2-unaudited-financial-statements.pdf',
  'assets/2024-q2-unaudited-financial-statements.pdf',
  'assets/2024-q2-unaudited-financial-statements.pdf',
  'assets/2024-q1-summary-unaudited-financial-statements.pdf',
  'assets/2024-q1-summary-unaudited-financial-statements.pdf',
  'assets/2024-q1-summary-unaudited-financial-statements.pdf',
  'assets/2024-q1-summary-unaudited-financial-statements.pdf',
  'assets/2023-audited-financial-statements.pdf',
  'assets/2023-audited-financial-statements.pdf',
  'assets/2023-audited-financial-statements.pdf',
  'assets/2023-audited-financial-statements.pdf',
  'assets/2023-summary-audited-financial-statements.pdf',
  'assets/2023-summary-audited-financia

In [10]:
financial_links['First_National_URL']

['/news-insight/financial-reports.html',
 '/news-insight/financial-reports.html',
 '/downloads/Ghana/financial-reports/2024/SummaryFinancialStatements_Sep2024.pdf',
 '/downloads/Ghana/financial-reports/2024/SummaryFinancialStatements_June2024.pdf',
 '/downloads/Ghana/financial-reports/2024/SummaryFinancialStatements_Mar2024.pdf',
 '/downloads/Ghana/financial-reports/2023/SummaryFinancialStatements_Dec2023.pdf',
 '/downloads/Ghana/financial-reports/2023/SummaryFinancialStatements_Sept2023.pdf',
 '/downloads/Ghana/financial-reports/2023/SummaryFinancialStatements_June2023.pdf',
 '/downloads/Ghana/financial-reports/2023/SummaryFinancialStatements_March2023.pdf',
 '/downloads/Ghana/financial-reports/2022/Financialpublication_December2022.pdf',
 '/downloads/Ghana/financial-reports/2022/Financialpublication_September2022.pdf',
 '/downloads/Ghana/financial-reports/2022/SummaryFinancialStatements_June2022.pdf',
 '/downloads/Ghana/financial-reports/2022/SummaryFinancialStatements_March2022.pdf'

In [11]:
cbg_base_url = 'https://cbg.com.gh/'

if "CBG_URL" in financial_links:
    cbg_links = financial_links["CBG_URL"]
    # Amend the links by prepending the base URL for relative links
    amended_links = [urljoin(cbg_base_url, link) for link in cbg_links]
    # Update the dictionary
    financial_links["CBG_URL"] = amended_links

In [12]:
First_National_base_url = 'https://www.firstnationalbank.com.gh/'
if "First_National_URL" in financial_links:
    First_National_links = financial_links["First_National_URL"]
    # Amend the links by prepending the base URL for relative links
    amended_links = [urljoin(First_National_base_url, link) for link in First_National_links]
    # Update the dictionary
    financial_links["First_National_URL"] = amended_links


In [13]:
SG_base_url = 'https://societegenerale.com.gh/'

if "SG_URL" in financial_links:
    SG_links = financial_links["SG_URL"]
    # Amend the links by prepending the base URL for relative links
    amended_links = [urljoin(SG_base_url, link) for link in SG_links]
    # Update the dictionary
    financial_links["SG_URL"] = amended_links



In [14]:
Zenith_base_url = 'https://www.zenithbank.com.gh/'

if "Zenith_URL" in financial_links:
    Zenith_links = financial_links["Zenith_URL"]
    # Amend the links by prepending the base URL for relative links
    amended_links = [urljoin(Zenith_base_url, link) for link in Zenith_links]
    # Update the dictionary
    financial_links["Zenith_URL"] = amended_links


In [15]:
Fidelity_base_url = 'https://www.fidelitybank.com.gh/'

if "Fidelity_URL" in financial_links:
    Fidelity_links = financial_links["Fidelity_URL"]
    # Amend the links by prepending the base URL for relative links
    amended_links = [urljoin(Fidelity_base_url, link) for link in Fidelity_links]
    # Update the dictionary
    financial_links["Fidelity_URL"] = amended_links

In [16]:
financial_links['CBG_URL']

['https://cbg.com.gh/assets/2024-q3-unaudited-financial-statements.pdf',
 'https://cbg.com.gh/assets/2024-q3-unaudited-financial-statements.pdf',
 'https://cbg.com.gh/assets/2024-q3-unaudited-financial-statements.pdf',
 'https://cbg.com.gh/assets/2024-q3-unaudited-financial-statements.pdf',
 'https://cbg.com.gh/assets/2024-q2-unaudited-financial-statements.pdf',
 'https://cbg.com.gh/assets/2024-q2-unaudited-financial-statements.pdf',
 'https://cbg.com.gh/assets/2024-q2-unaudited-financial-statements.pdf',
 'https://cbg.com.gh/assets/2024-q2-unaudited-financial-statements.pdf',
 'https://cbg.com.gh/assets/2024-q1-summary-unaudited-financial-statements.pdf',
 'https://cbg.com.gh/assets/2024-q1-summary-unaudited-financial-statements.pdf',
 'https://cbg.com.gh/assets/2024-q1-summary-unaudited-financial-statements.pdf',
 'https://cbg.com.gh/assets/2024-q1-summary-unaudited-financial-statements.pdf',
 'https://cbg.com.gh/assets/2023-audited-financial-statements.pdf',
 'https://cbg.com.gh/ass

In [17]:
#extract the actual text from the website
keyword = "Non-performing loan"
#pattern = rf"(.*{keyword}.*(?:\n.*){{0,3}})"
text = []
print("count= ",len(sep))
for link in sep:
    with pdfplumber.open(link) as pdf:
        page = pdf.pages[-1]
        read_text = page.extract_text()
        text.append(read_text)
        lines = read_text.splitlines()
        print(link)
        for i, line in enumerate(lines):
            if keyword in line:
                for j in range(i,min(i+3,len(lines))):
                               print(lines[j])
                print("-"*40)    

NameError: name 'sep' is not defined

In [ ]:
def extract_npl_from_pdf(pdf_url):
    # Download the PDF
    response = requests.get(pdf_url)
    pdf_filename = urlparse(pdf_url).path.split('/')[-1]

    # Write the content to a file locally
    with open(pdf_filename, 'wb') as f:
        f.write(response.content)

    # Extract text from the PDF using pdfplumber
    with pdfplumber.open(pdf_filename) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()

    # Search for the Non-performing loan ratio using a regex
    match = re.compile(r'non[-\s]?performing\s+loan[\s]*[:\-\s]*([\d\.]+%?)', text, re.IGNORECASE)
    npl_ratio = match.group(1) if match else None

    return pdf_filename, npl_ratio

# Function to process each bank's URL and extract NPL info
def process_banks_and_create_new_library(bank_dict):
    # Create a new dictionary to store the results
    new_bank_dict = {}

    # Loop through each bank and URL in the original dictionary
    for bank_name, bank_url in bank_dict.items():
        # Fetch the page content of the bank's URL
        response = requests.get(bank_url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Look for PDF links in the page
        pdf_link = None
        for link in soup.find_all('a', href=True):
            if link['href'].lower().endswith('.pdf'):
                pdf_link = link['href']
                break

        # If a PDF link is found, process the PDF
        if pdf_link:
            pdf_filename, npl_ratio = extract_npl_from_pdf(pdf_link)
            # Add the result to the new dictionary
            new_bank_dict[bank_name] = {
                'bank_name': bank_name,
                'pdf_filename': pdf_filename,
                'npl_ratio': npl_ratio
            }
        else:
            # If no PDF link is found, add None values in the new dictionary
            new_bank_dict[bank_name] = {
                'bank_name': bank_name,
                'pdf_filename': None,
                'npl_ratio': None
            }

    return new_bank_dict

# Example input dictionary with bank names and URLs
#bank_urls = {
#   "Bank1": "https://example.com/bank1-report.html",  # Example URL
#  "Bank2": "https://example.com/bank2-report.html",  # Example URL
#}

# Process the banks and create a new library (new dictionary)
new_banks = process_banks_and_create_new_library(financial_links)

# Output the new dictionary
print(new_banks)

In [ ]:
# Function to download and parse the PDF to extract NPL ratio (including OCR for image-based pages)
def extract_npl_from_pdf(pdf_url):
    # Download the PDF
    response = requests.get(pdf_url)
    pdf_filename = urlparse(pdf_url).path.split('/')[-1]

    # Write the content to a file locally
    with open(pdf_filename, 'wb') as f:
        f.write(response.content)

    # Extract text from the PDF using pdfplumber for text-based pages
    text = ""
    with pdfplumber.open(pdf_filename) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
            else:
                # If no text, it's likely an image, so we need OCR
                text += extract_text_from_image(page)

    # Search for the Non-performing loan ratio using a regex
    match = re.search(r'non[-\s]?performing\s+loan[\s]*[:\-\s]*([\d\.]+%?)', text, re.IGNORECASE)
    npl_ratio = match.group(1) if match else None

    return pdf_filename, npl_ratio

# Function to extract text from image-based PDF pages using OCR
def extract_text_from_image(page):
    # Convert the page to an image using pdf2image
    images = convert_from_path(page.stream.name, first_page=page.page_number, last_page=page.page_number)
    image = images[0]
    
    # Use pytesseract to extract text from the image
    text = pytesseract.image_to_string(image)
    return text

# Function to process each bank's URL and extract NPL info
def process_banks_and_create_new_library(bank_dict):
    # Create a new dictionary to store the results
    new_bank_dict = {}

    # Loop through each bank and URL in the original dictionary
    for bank_name, pdf_url in bank_dict.items():
        # Process the PDF to extract the filename and NPL ratio
        pdf_filename, npl_ratio = extract_npl_from_pdf(pdf_url)
        
        # Add the result to the new dictionary
        new_bank_dict[bank_name] = {
            'bank_name': bank_name,
            'pdf_filename': pdf_filename,
            'npl_ratio': npl_ratio
        }

    return new_bank_dict

# Example input dictionary with bank names and PDF URLs
#bank_urls = {
   # "Bank1": "https://example.com/bank1-report.pdf",  # Example PDF URL
   # "Bank2": "https://example.com/bank2-report.pdf",  # Example PDF URL
#}

# Process the banks and create a new library (new dictionary)
new_banks = process_banks_and_create_new_library(financial_links)

# Output the new dictionary
print(new_banks)

In [ ]:
#URLs = [CBG_URL, GT_URL, FBN_URL, First_National_URL, Cal_URL, SG_URL, Stanbic_URL, Zenith_URL, Fidelity_URL, Ecobank_URL]

In [ ]:
#CBG (class = block in the a tag) 
#
non-performing loan ratio.*?([/d.]+%